In [2]:
import numpy as np

In [3]:
def reduce_path(org_path):
    aa, bb = org_path
    largest_cand = max(abs(aa), abs(bb))
    for cand in range(largest_cand, 0, -1):
        if aa//cand == aa/cand and bb//cand == bb/cand:
            return (aa//cand, bb//cand)
    return (aa, bb)

In [4]:
ex_inp = """.#..#
.....
#####
....#
...##"""

In [5]:
ex_inp2 = """......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####"""

ex_inp3 = """#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###."""

ex_inp_large = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""

In [6]:
def get_greatest_num(inp_string):
    location_list = []
    inp_rows = inp_string.split('\n')
    width = len(inp_rows[0])
    for ii in range(len(inp_rows)):
        for jj in range(width):
            if inp_rows[ii][jj] == '#':
                location_list.append((jj, ii))

    greatest_num = 0
    greatest_num_loc = (0, 0)
    greatest_num_idx = -1
    for loc_idx in range(len(location_list)):
        loc_aa, loc_bb = location_list[loc_idx]
        other_locs = location_list[:loc_idx]+location_list[loc_idx+1:]
    #     print((loc_aa, loc_bb))
    #     print(other_locs)
        paths = [(aa-loc_aa, bb-loc_bb) for (aa,bb) in other_locs]
        reduced_paths = [reduce_path(pp) for pp in paths]
        num_seen = len(set(reduced_paths))
    #     print(num_seen)
        if num_seen > greatest_num:
            greatest_num = num_seen
            greatest_num_loc = (loc_aa, loc_bb)
            greatest_num_idx = loc_idx

    print(greatest_num)
    other_locs_greatest = location_list[:greatest_num_idx]\
        +location_list[greatest_num_idx+1:]
    return greatest_num_loc, other_locs_greatest

In [7]:
# Run examples
get_greatest_num(ex_inp2)
get_greatest_num(ex_inp3)
ast_loc_ex, other_locs_ex = get_greatest_num(ex_inp_large)

33
35
210


In [8]:
ast_loc_ex

(11, 13)

In [9]:
# Part one input
puzzle_inp = """#...##.####.#.......#.##..##.#.
#.##.#..#..#...##..##.##.#.....
#..#####.#......#..#....#.###.#
...#.#.#...#..#.....#..#..#.#..
.#.....##..#...#..#.#...##.....
##.....#..........##..#......##
.##..##.#.#....##..##.......#..
#.##.##....###..#...##...##....
##.#.#............##..#...##..#
###..##.###.....#.##...####....
...##..#...##...##..#.#..#...#.
..#.#.##.#.#.#####.#....####.#.
#......###.##....#...#...#...##
.....#...#.#.#.#....#...#......
#..#.#.#..#....#..#...#..#..##.
#.....#..##.....#...###..#..#.#
.....####.#..#...##..#..#..#..#
..#.....#.#........#.#.##..####
.#.....##..#.##.....#...###....
###.###....#..#..#.....#####...
#..##.##..##.#.#....#.#......#.
.#....#.##..#.#.#.......##.....
##.##...#...#....###.#....#....
.....#.######.#.#..#..#.#.....#
.#..#.##.#....#.##..#.#...##..#
.##.###..#..#..#.###...#####.#.
#...#...........#.....#.......#
#....##.#.#..##...#..####...#..
#.####......#####.....#.##..#..
.#...#....#...##..##.#.#......#
#..###.....##.#.......#.##...##"""

In [10]:
# Part one solution
ast_location, other_locs = get_greatest_num(puzzle_inp)

288


In [11]:
# reduced_paths, when multiplied out again, will give 
#    the set of all points that will be vaporized in the first round
# the angle can be calculated as tan(y/x), gives ordering
# other_locs will need to be updated when vaporization occurs

In [12]:
def reconstruct_loc(reduced_path, ast_loc, other_locs):
    other_locs_set = set(other_locs)
    (loc_aa, loc_bb) = ast_loc
    (red_aa, red_bb) = reduced_path
    for ii in range(100):
        test_loc = (loc_aa+red_aa*ii, loc_bb+red_bb*ii)
        if test_loc in other_locs_set:
            return test_loc
    

In [13]:
def take_arctan(aa, bb_pp):
    bb = - bb_pp
    if bb == 0:
        if aa >= 0:
            return np.pi/2
        else:
            return 3*np.pi/2
        return 0
    if aa >= 0:
        if bb >= 0:
            return np.arctan(aa/bb)
        else:
            return np.pi + np.arctan(aa/bb)
    else:
        if bb >= 0:
            return 2*np.pi - np.arctan(np.abs(aa)/bb)
        else:
            return np.pi + np.arctan(aa/bb)
            

In [22]:
def track_vaporisation(ast_loc, other_locs, of_interest):
    currently_vaporised = 0
    (loc_aa, loc_bb) = ast_loc
    other_locs_var = [pp for pp in other_locs]
    while len(other_locs_var) > 0:
        print('%s\t%s' %(currently_vaporised,len(other_locs_var)))
        paths = [(aa-loc_aa, bb-loc_bb) for (aa,bb) in other_locs_var]
        reduced_paths = [reduce_path(pp) for pp in paths]
        reduced_detected = set(reduced_paths)
#         print(reduced_detected)
#         print(len(reduced_detected))
        if currently_vaporised + len(reduced_detected) < of_interest:
            # Remove these reconstructed locations from other_locs_var
            vaporised_locations = [reconstruct_loc(rr, ast_loc, other_locs_var) 
                                   for rr in reduced_detected]
            other_locs_var = [oo for oo in other_locs_var 
                              if oo not in vaporised_locations]
            currently_vaporised = currently_vaporised + len(reduced_detected)
        else:
            angles = [(take_arctan(aa,bb), (aa, bb)) for (aa,bb) in reduced_detected]
            angles.sort(key=lambda tt:  tt[0])
            interest_idx = of_interest - currently_vaporised - 1
            print(reconstruct_loc(angles[interest_idx][1], ast_loc, other_locs_var))
            
#             print([(reconstruct_loc(loc[1], ast_loc, other_locs_var), loc[0]) for loc in angles])
#             print([reconstruct_loc(loc[1], ast_loc, other_locs_var) for loc in angles])

            #             print(angles)
            break

#     print(currently_vaporised)
#     return greatest_num_loc

In [23]:
track_vaporisation(ast_loc_ex, other_locs_ex, of_interest=299)

0	299
210	89
253	46
270	29
278	21
284	15
289	10
292	7
294	5
296	3
297	2
298	1
(11, 1)


In [25]:
# Part two solution
track_vaporisation(ast_location, other_locs, of_interest=200);

0	359
(6, 16)
